In [4]:
!pip install seaborn catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Import all regression models covered in class
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
    StackingRegressor,
    VotingRegressor
)
from sklearn.neural_network import MLPRegressor

import warnings
warnings.filterwarnings('ignore')

# Loading the Raw Data

We pull in the raw test and train cattle datasets. This step is about making sure the files are loaded correctly, checking the shapes, skimming the first few rows, and confirming the data types of the dataset.

In [10]:
# Load test and train cattle datasets
train_data = pd.read_csv('sample_data/cattle_data_train.csv')
test_data = pd.read_csv('sample_data/cattle_data_test.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print()

# Display first few rows of the data
print("First few rows of training data:")
print(train_data.head())
print()

# Display data types
print("Data types:")
print(train_data.dtypes)
print()

# Inspect column names
print("Column names:")
print(train_data.columns.tolist())
print()


Training data shape: (210000, 36)
Test data shape: (40000, 35)

First few rows of training data:
       Cattle_ID     Breed   Climate_Zone Management_System  Age_Months  \
0  CATTLE_133713  Holstein       Tropical         Intensive         114   
1  CATTLE_027003  Holstein           Arid             Mixed         136   
2  CATTLE_122459  Holstein       Tropical    Semi_Intensive          64   
3  CATTLE_213419    Jersey  Mediterranean         Intensive          58   
4  CATTLE_106260  Guernsey    Subtropical         Intensive          84   

   Weight_kg  Parity Lactation_Stage  Days_in_Milk      Feed_Type  ...  \
0      544.8       4             Mid            62   Concentrates  ...   
1      298.9       4             Mid           213  Crop_Residues  ...   
2      336.6       4            Late            16            Hay  ...   
3      370.5       1           Early           339  Crop_Residues  ...   
4      641.5       6           Early           125     Mixed_Feed  ...   

   BVD_

# Data Exploration

In [ ]:
# Shiv Testing
print("Train Milking_Interval_hrs:")
series = train_data['Milking_Interval_hrs']
print(series)


print("\nBasic statistics:")
print(f"  count:   {series.count()}")
print(f"  missing: {series.isna().sum()}")
print(f"  unique:  {series.nunique()}")
print(f"  min:     {series.min()}")
print(f"  max:     {series.max()}")
print(f"  mean:    {series.mean():.4f}")
print(f"  median:  {series.median():.4f}")
print(f"  std:     {series.std():.4f}")
modes = series.mode().tolist()
print(f"  mode(s): {modes}")

print("\nUnique values:")
print(sorted(series.dropna().unique().tolist()))

print("\nQuantiles:")
quantiles = series.quantile([0.05, 0.25, 0.5, 0.75, 0.95])
for q, v in quantiles.items():
    print(f"  {int(q*100):>2}th: {v}")

print("\nTop 10 most frequent values:")
print(series.value_counts().head(10))


Train Milking_Interval_hrs:
0         12
1         12
2         12
3         24
4         12
          ..
209995    12
209996    12
209997    24
209998    12
209999    12
Name: Milking_Interval_hrs, Length: 210000, dtype: int64

Basic statistics:
  count:   210000
  missing: 0
  unique:  4
  min:     6
  max:     24
  mean:    12.3024
  median:  12.0000
  std:     4.2990
  mode(s): [12]

Unique values:
[6, 8, 12, 24]

Quantiles:
   5th: 8.0
  25th: 12.0
  50th: 12.0
  75th: 12.0
  95th: 24.0

Top 10 most frequent values:
Milking_Interval_hrs
12    147167
8      31399
24     20984
6      10450
Name: count, dtype: int64


To get a feel for the dataset, we checked the structure of the data to spot missing values and understand how the target var (Milk_Yield_L) behaves. We also look at the distribution of the target variable using a histogram and box plot, helping us understand how the data is skewed and whether or not outliers might cause any issues during model training.

In [11]:

# Basic Dataset Info
print("Dataset Overview")
print("-" * 80)
print(train_data.describe())
print()

# Check for Missing Values
print("Missing Values Analysis")
print("-" * 80)
missing_counts = train_data.isnull().sum()
missing_pct = 100 * train_data.isnull().sum() / len(train_data)
missing_table = pd.DataFrame({
    'Missing_Count': missing_counts,
    'Percentage': missing_pct
})
missing_table = missing_table[missing_table['Missing_Count'] > 0].sort_values(
    'Percentage', ascending=False
)
print(missing_table)
print()

# Visualize missing data pattern
if len(missing_table) > 0:
    plt.figure(figsize=(12, 6))
    missing_table['Percentage'].plot(kind='barh')
    plt.xlabel('Percentage Missing')
    plt.title('Missing Data by Feature')
    plt.tight_layout()
    plt.savefig('missing_data.png')
    plt.close()
    print("✓ Missing data visualization saved as 'missing_data.png'")
print()

# Target Variable Distribution
print("1.3 Target Variable (Milk_Yield_L) Distribution")
print("-" * 80)
print(f"Mean:   {train_data['Milk_Yield_L'].mean():.2f} L")
print(f"Median: {train_data['Milk_Yield_L'].median():.2f} L")
print(f"Std:    {train_data['Milk_Yield_L'].std():.2f} L")
print(f"Min:    {train_data['Milk_Yield_L'].min():.2f} L")
print(f"Max:    {train_data['Milk_Yield_L'].max():.2f} L")
print(f"Skewness: {train_data['Milk_Yield_L'].skew():.2f}")
print()

# Plot distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].hist(train_data['Milk_Yield_L'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Milk Yield (L)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Milk Yield')
axes[0].axvline(train_data['Milk_Yield_L'].mean(), color='r', linestyle='--', label='Mean')
axes[0].axvline(train_data['Milk_Yield_L'].median(), color='g', linestyle='--', label='Median')
axes[0].legend()

axes[1].boxplot(train_data['Milk_Yield_L'])
axes[1].set_ylabel('Milk Yield (L)')
axes[1].set_title('Box Plot of Milk Yield')
plt.tight_layout()
plt.savefig('target_distribution.png')
plt.close()
print("✓ Target distribution saved as 'target_distribution.png'")
print()


Dataset Overview
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210000 entries, 0 to 209999
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Cattle_ID                210000 non-null  object 
 1   Breed                    210000 non-null  object 
 2   Climate_Zone             210000 non-null  object 
 3   Management_System        210000 non-null  object 
 4   Age_Months               210000 non-null  int64  
 5   Weight_kg                210000 non-null  float64
 6   Parity                   210000 non-null  int64  
 7   Lactation_Stage          210000 non-null  object 
 8   Days_in_Milk             210000 non-null  int64  
 9   Feed_Type                210000 non-null  object 
 10  Feed_Quantity_kg         199519 non-null  float64
 11  Feeding_Frequency        210000 non-null  int64  
 12  Water_Intake_L  

# Feature Types, Dates, Correlations

To understand what features we're dealing with, we separate the categorical and numerical columns. The dataset has vaccine-related columns — we grouped these in with the categorical.

For the numeric columns, we check if there are any redundant features. In this dataset, Feed_Quantity_kg and Feed_Quantity_lb represent the same thing but in different units, so we confirmed the ratio to be sure.

We also observed the date column and converted it to a proper datetime. This allows us to more easily extract what we want from the date later on (during feature engineering).

Finally, we ran a correlation analysis to see what numeric features actually relate to milk production. This gives us an idea of what variables might be useful and which ones might be noise.

In [13]:
# Identify Numeric vs Categorical Features
print("Feature Type Identification")
print("-" * 80)

# Define categorical and numeric features based on the dataset
categorical_features = [
    'Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage',
    'Feed_Type', 'Mastitis'
]

# Find vaccine columns (pattern: [Disease]_Vaccine)
vaccine_cols = [col for col in train_data.columns if '_Vaccine' in col]
print(f"Found {len(vaccine_cols)} vaccine columns: {vaccine_cols}")
categorical_features.extend(vaccine_cols)

# Numeric features (excluding ID, target, and date)
numeric_features = [
    'Age_Months', 'Weight_kg', 'Parity', 'Days_in_Milk', 'Feed_Quantity_kg',
    'Feeding_Frequency', 'Water_Intake_L', 'Walking_Distance_km',
    'Grazing_Duration_hrs', 'Rumination_Time_hrs', 'Resting_Hours',
    'Ambient_Temperature_C', 'Humidity_percent', 'Housing_Score',
    'Previous_Week_Avg_Yield', 'Body_Condition_Score', 'Milking_Interval_hrs',
    'Feed_Quantity_lb'
]

print(f"\nNumeric features ({len(numeric_features)}):")
for feat in numeric_features:
    print(f"  • {feat}")
print(f"\nCategorical features ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  • {feat}")
print()

# Check for duplicate Feed_Quantity columns
print("Checking Feed Quantity Columns")
print("-" * 80)
# Check conversion factor
ratio = (train_data['Feed_Quantity_lb'] / train_data['Feed_Quantity_kg']).mean()
print(f"Average lb/kg ratio: {ratio:.4f} (expected: 2.20462)")
print()

# Date Feature Analysis
print("Date Feature Analysis")
print("-" * 80)
train_data['Date'] = pd.to_datetime(train_data['Date'])
print(f"Date range: {train_data['Date'].min()} to {train_data['Date'].max()}")

print()

# Correlation Analysis (for numeric features)
print("Correlation Analysis with Target")
print("-" * 80)

# Calculate correlations with Milk_Yield_L
correlations = train_data[numeric_features + ['Milk_Yield_L']].corr()['Milk_Yield_L'].drop('Milk_Yield_L')
correlations = correlations.sort_values(ascending=False)
print("Top 10 features most correlated with Milk_Yield_L:")
print(correlations.head(10))
print()
print("Bottom 10 features (least correlated or negatively correlated):")
print(correlations.tail(10))
print()


Feature Type Identification
--------------------------------------------------------------------------------
Found 8 vaccine columns: ['FMD_Vaccine', 'Brucellosis_Vaccine', 'HS_Vaccine', 'BQ_Vaccine', 'Anthrax_Vaccine', 'IBR_Vaccine', 'BVD_Vaccine', 'Rabies_Vaccine']

Numeric features (18):
  • Age_Months
  • Weight_kg
  • Parity
  • Days_in_Milk
  • Feed_Quantity_kg
  • Feeding_Frequency
  • Water_Intake_L
  • Walking_Distance_km
  • Grazing_Duration_hrs
  • Rumination_Time_hrs
  • Resting_Hours
  • Ambient_Temperature_C
  • Humidity_percent
  • Housing_Score
  • Previous_Week_Avg_Yield
  • Body_Condition_Score
  • Milking_Interval_hrs
  • Feed_Quantity_lb

Categorical features (14):
  • Breed
  • Climate_Zone
  • Management_System
  • Lactation_Stage
  • Feed_Type
  • Mastitis
  • FMD_Vaccine
  • Brucellosis_Vaccine
  • HS_Vaccine
  • BQ_Vaccine
  • Anthrax_Vaccine
  • IBR_Vaccine
  • BVD_Vaccine
  • Rabies_Vaccine

Checking Feed Quantity Columns
-------------------------------------

# Categorical Features + Outlier Checks

We looked at the categorical columns to understand how many unique values each column has. This helps us later when deciding between one-hot encoding vs frequency encoding vs dropping columns that barely vary.

For numeric features, outlier detection is key. We ran an IQR-based outlier detection, which gave us an idea of which features have extreme values. For every feature with outliers, we print out how many points fall outside the acceptable range → we now know where to focus during preprocessing.

In [14]:
# Categorical Feature Analysis
print("Categorical Feature Analysis")
print("-" * 80)
for cat_feat in categorical_features[:5]:  # Show first 5
    if cat_feat in train_data.columns:
        n_unique = train_data[cat_feat].nunique()
        print(f"{cat_feat}: {n_unique} unique values")
        print(f"  Values: {train_data[cat_feat].value_counts().head().to_dict()}")
print()

# Outlier Detection
print("Outlier Detection Using IQR Method")
print("-" * 80)

def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method."""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("Outliers in numeric features:")
for feature in numeric_features:
    if feature in train_data.columns:
        n_outliers, lower, upper = detect_outliers_iqr(train_data, feature)
        if n_outliers > 0:
            pct = 100 * n_outliers / len(train_data)
            print(f"{feature:30} {n_outliers:5} outliers ({pct:5.2f}%) [bounds: {lower:.2f} to {upper:.2f}]")
print()


Categorical Feature Analysis
--------------------------------------------------------------------------------
Breed: 7 unique values
  Values: {'Holstein': 104775, 'Jersey': 42183, 'Guernsey': 31672, 'Brown Swiss': 31155, 'Holstien': 112}
Climate_Zone: 6 unique values
  Values: {'Temperate': 35224, 'Tropical': 35062, 'Mediterranean': 34994, 'Arid': 34954, 'Subtropical': 34937}
Management_System: 5 unique values
  Values: {'Intensive': 42225, 'Pastoral': 42126, 'Extensive': 41973, 'Semi_Intensive': 41906, 'Mixed': 41770}
Lactation_Stage: 3 unique values
  Values: {'Mid': 83895, 'Early': 63203, 'Late': 62902}
Feed_Type: 8 unique values
  Values: {'Dry_Fodder': 26558, 'Pasture_Grass': 26305, 'Crop_Residues': 26278, 'Concentrates': 26231, 'Mixed_Feed': 26229}

Outlier Detection Using IQR Method
--------------------------------------------------------------------------------
Outliers in numeric features:
Feed_Quantity_kg                 674 outliers ( 0.32%) [bounds: 1.14 to 22.85]
Water_In

# Data Cleaning

We first removed the redundant column of Feed_Quantity_lb, since it represents the same information as Feed_Quantity_kg, except in pounds. We then processed the Date column (it was converted to datetime during data exploration). We extract the month, dayOfWeek, and quarter from the date. We also add cyclical month features (sin / cos — to transform the month number into two new features). This is because seasons loop around, and models don't naturally understand that December is next to January.

For the missing values, we filled in the numeric columns using the training means. For outliers, instead of deleting the rows, we capped them based on 3*IQR. We did this for all columns with outliers except Milking_Interval_hrs: we skipped this column because it had an unusual distribution.

Next, we handled the categorical variables. For binary features, we used simple label encoding, and for other low-cardinality features, we applied one-hot encoding. Importantly, we always fit the encodings on the training set first and then aligned the test set to match those columns.

Finally, we removed low-variance features (columns where 95%+ of values were the same) because they don't help the model. We also ran a safety check at the very end to make sure there were no more missing values in the data (for test and train).

The result of this data cleaning cell is two fully cleaned dataframes that are safe to feed into a model without worrying about missing data, redundant/useless features, or inconsistent preprocessing.

In [18]:
# ============================================================================
# PHASE 2: DATA CLEANING
# ============================================================================
print("="*80)
print("PHASE 2: DATA CLEANING")
print("="*80)
print()

def clean_data(train_df, test_df):
    """
    Comprehensive data cleaning function for cattle milk yield dataset.

    Args:
        train_df: Training DataFrame (has target)
        test_df:  Test DataFrame (no target)

    Returns:
        (train_clean, test_clean): cleaned DataFrames
    """
    train_clean = train_df.copy()
    test_clean = test_df.copy()

    print("Cleaning training and test data (no leakage)...")
    print("-" * 80)

    # Drop redundant Feed_Quantity_lb (duplicate of Feed_Quantity_kg)
    print("Removing Redundant Features")
    print("-" * 40)
    for df_name, df_clean in [("training", train_clean), ("test", test_clean)]:
          df_clean.drop('Feed_Quantity_lb', axis=1, inplace=True)
    print()

    # Handle Date Feature - Extract temporal features
    print("Processing Date Feature")
    print("-" * 40)
    for df_name, df_clean in [("training", train_clean), ("test", test_clean)]:
        if 'Date' in df_clean.columns:
            df_clean['Date'] = pd.to_datetime(df_clean['Date'])

            # Extract useful temporal features
            df_clean['Month'] = df_clean['Date'].dt.month
            df_clean['DayOfWeek'] = df_clean['Date'].dt.dayofweek
            df_clean['Quarter'] = df_clean['Date'].dt.quarter

            # Create cyclical features for month (since December is close to January)
            df_clean['Month_sin'] = np.sin(2 * np.pi * df_clean['Month'] / 12)
            df_clean['Month_cos'] = np.cos(2 * np.pi * df_clean['Month'] / 12)

            print(f"✓ ({df_name}) Extracted: Month, DayOfWeek, Quarter")
            print(f"✓ ({df_name}) Created cyclical features: Month_sin, Month_cos")

            # Drop original Date column (not useful for ML models directly)
            df_clean.drop('Date', axis=1, inplace=True)
            print(f"✓ ({df_name}) Dropped original Date column")
    print()

    # Handle Missing Values (NUMERIC) - FIT ON TRAIN, APPLY TO TEST
    print("Handling Missing Values (Numeric)")
    print("-" * 40)

    numeric_means = {}  # store means from training only

    for col in numeric_features:
        if col in train_clean.columns:
            missing_pct = 100 * train_clean[col].isnull().sum() / len(train_clean)
            if train_clean[col].isnull().sum() > 0:
                mean_val = train_clean[col].mean()
                numeric_means[col] = mean_val
                train_clean[col].fillna(mean_val, inplace=True)
                print(f"  (train) {col:30} {missing_pct:5.2f}% missing → imputed with median ({mean_val:.2f})")

                if col in test_clean.columns:
                    test_clean[col].fillna(mean_val, inplace=True)
                    print(f"  (test)  {col:30} imputed with train median ({mean_val:.2f})")
            else:
                # still store for consistency
                numeric_means[col] = train_clean[col].median()

    print()

    # Handle Outliers (Cap rather than remove) - FIT BOUNDS ON TRAIN, APPLY TO TEST
    print("Handling Outliers (Capping Method)")
    print("-" * 40)

    outlier_count = 0
    for col in numeric_features:
        if col in train_clean.columns:
            if col == 'Milking_Interval_hrs':
                print(f"  (train/test) {col:30} skipped outlier capping")
                continue

            Q1 = train_clean[col].quantile(0.25)
            Q3 = train_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR  # Use 3*IQR for conservative capping
            upper_bound = Q3 + 3 * IQR

            n_lower_train = (train_clean[col] < lower_bound).sum()
            n_upper_train = (train_clean[col] > upper_bound).sum()

            if n_lower_train > 0 or n_upper_train > 0:
                train_clean[col] = train_clean[col].clip(lower_bound, upper_bound)
                outlier_count += 1
                print(f"  (train) {col:30} capped {n_lower_train} lower, {n_upper_train} upper outliers")

            # apply same bounds to test (if column exists)
            if col in test_clean.columns:
                test_clean[col] = test_clean[col].clip(lower_bound, upper_bound)

    if outlier_count == 0:
        print("  No significant outliers detected (using 3*IQR threshold)")
    print()

    # Encode Categorical Variables - FIT ON TRAIN, APPLY TO TEST
    print("Encoding Categorical Variables")
    print("-" * 40)

    for col in categorical_features:
        if col in train_clean.columns:
            n_unique = train_clean[col].nunique()

            # Binary: Simple label encoding (0/1) – fit on train
            if n_unique <= 2:
                le = LabelEncoder()
                train_clean[col] = train_clean[col].astype(str)
                le.fit(train_clean[col])
                mapping = {cls: idx for idx, cls in enumerate(le.classes_)}

                train_clean[col] = train_clean[col].map(mapping).astype(int)
                print(f"  (train) {col:30} Binary → Label encoded")

                # Apply same mapping to test; unseen categories → -1
                if col in test_clean.columns:
                    test_clean[col] = test_clean[col].astype(str).map(mapping).fillna(-1).astype(int)
                    print(f"  (test)  {col:30} Binary → Label encoded with train mapping")

            # Low cardinality: One-hot encoding – fit categories on train
            elif n_unique < 10:
                # TRAIN dummies
                dummies_train = pd.get_dummies(train_clean[col], prefix=col, drop_first=True)
                train_clean = pd.concat([train_clean, dummies_train], axis=1)
                train_clean.drop(col, axis=1, inplace=True)

                print(f"  (train) {col:30} {n_unique} categories → One-hot encoded")

                # TEST dummies with same columns
                if col in test_clean.columns:
                    dummies_test = pd.get_dummies(test_clean[col], prefix=col, drop_first=True)
                    # Align to training dummy columns
                    dummies_test = dummies_test.reindex(columns=dummies_train.columns, fill_value=0)
                    test_clean = pd.concat([test_clean, dummies_test], axis=1)
                    test_clean.drop(col, axis=1, inplace=True)
                    print(f"  (test)  {col:30} aligned one-hot columns to train")
    print()

    # Remove Low-Variance Features – DECIDE ON TRAIN, DROP IN BOTH
    print("Removing Low-Variance Features")
    print("-" * 40)

    removed_features = []
    exclude_cols = ['Cattle_ID', 'Farm_ID', 'Milk_Yield_L']

    for col in train_clean.select_dtypes(include=[np.number]).columns:
        if col not in exclude_cols:
            if len(train_clean[col].value_counts()) > 0:
                mode_freq = train_clean[col].value_counts().iloc[0] / len(train_clean)
                if mode_freq > 0.95:
                    removed_features.append(col)

    # Drop same features from both train and test
    if removed_features:
        train_clean.drop(columns=removed_features, axis=1, inplace=True, errors='ignore')
        test_clean.drop(columns=removed_features, axis=1, inplace=True, errors='ignore')
        print(f"  Removed {len(removed_features)} low-variance features (from train, applied to test too):")
        for feat in removed_features:
            print(f"    • {feat}")
    else:
        print("  No low-variance features detected")
    print()

    # Final safety check - fill any remaining NaN values
    print("Final Safety Check for NaN Values")
    print("-" * 40)

    # Do this for both train and test, but use train medians where possible
    for df_name, df_clean in [("training", train_clean), ("test", test_clean)]:
        remaining_nan = df_clean.isnull().sum().sum()
        if remaining_nan > 0:
            print(f"  ({df_name}) WARNING: Found {remaining_nan} remaining NaN values")
            print(f"  ({df_name}) → Filling remaining numeric NaN with TRAIN medians / 0")

            # For numeric columns, fill with medians based on training data
            numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if df_clean[col].isnull().any():
                    if col in train_clean.columns:
                        # median from train distribution
                        median_val = train_clean[col].median()
                    else:
                        # column not in train (e.g., test-only) – just use 0
                        median_val = 0
                    df_clean[col].fillna(median_val, inplace=True)

            # For any remaining non-numeric NaN, fill with 'Unknown'
            non_num_cols = df_clean.select_dtypes(exclude=[np.number]).columns
            df_clean[non_num_cols] = df_clean[non_num_cols].fillna('Unknown')
        else:
            print(f"  ({df_name}) ✓ No NaN values detected")

        # Check for infinite values in each df separately
        inf_count = np.isinf(df_clean.select_dtypes(include=[np.number]).values).sum()
        if inf_count > 0:
            print(f"  ({df_name}) WARNING: Found {inf_count} infinite values → Replacing with finite caps")
            df_clean.replace([np.inf, -np.inf], [1e10, -1e10], inplace=True)

    print()
    return train_clean, test_clean

# Apply cleaning to both train and test
train_cleaned, test_cleaned = clean_data(train_data, test_data)

print(f"✓ Training data shape after cleaning: {train_cleaned.shape}")
print(f"✓ Test data shape after cleaning: {test_cleaned.shape}")
print()

print(
    f"Columns — train ({len(train_cleaned.columns)}): {list(train_cleaned.columns)}\n"
    f"Columns — test ({len(test_cleaned.columns)}): {list(test_cleaned.columns)}"
)


PHASE 2: DATA CLEANING

Cleaning training and test data (no leakage)...
--------------------------------------------------------------------------------
Removing Redundant Features
----------------------------------------

Processing Date Feature
----------------------------------------
✓ (training) Extracted: Month, DayOfWeek, Quarter
✓ (training) Created cyclical features: Month_sin, Month_cos
✓ (training) Dropped original Date column
✓ (test) Extracted: Month, DayOfWeek, Quarter
✓ (test) Created cyclical features: Month_sin, Month_cos
✓ (test) Dropped original Date column

Handling Missing Values (Numeric)
----------------------------------------
  (train) Feed_Quantity_kg                4.99% missing → imputed with median (12.01)
  (test)  Feed_Quantity_kg               imputed with train median (12.01)
  (train) Housing_Score                   2.99% missing → imputed with median (0.65)
  (test)  Housing_Score                  imputed with train median (0.65)

Handling Outliers (Ca

# Feature Engineering

Now that our data is clean, we started adding features that improve the model's accuracy by adding in some domain intuition about dairy cows and milk production.

First, we created ratio features for productivity and efficiency, like feed per unit weight, water per unit weight, and milk yield per kg of feed. These ratio features are meant to capture how efficiently each cow converts its feed and water into milk, rather than just using the raw totals.

Next, we added in activity and health-related features using walking distance, grazing duration, rumination time, resting hours, body condition score, etc. More balanced activity and rest, plus better body condition = healthier, more productive cows.

We also focused on lactation-specific features. We added these features: whether or not the cow is in its “peak” lactation window (cows produce the most milk at around 60 days), how often it's being milked per day based on the milking interval, and parity-related information (first-time vs experienced mothers — milk production changes with each lactation).

For environment and age, we created simple indicators for extreme temperatures (based on the Ambient_Temperature_C column), plus age groups (young, prime, senior) and an approximate average age at calving. These features are meant to capture long-term effects like aging and environmental stress on milk yield.

We also added some feed and nutrition features like feed per meal and feed-to-water ratio, to reflect how feeding routines might affect production. After this, we scanned our data for highly skewed distributions and then applied log transforms on them (to stabilize).

Finally, we included some interaction features (like weight × age, feed × water, parity × days in milk). These features let the model pick up on some important combinations of these features that might not be so obvious from the raw features alone.

All of this is done on the test and train data.

Links:

https://www.sciencedirect.com/science/article/pii/S2666910225000924#:~:text=In%20conclusion%2C%20cow%20lactation%20milk,linearly%20thereafter%20with%20parity%20number.

https://extension.psu.edu/evaluating-milk-peak-and-persistency-using-dhia-data-part-2#:~:text=Cows%20usually%20peak%20around%2045,milk%20over%20her%20entire%20lactation.


In [20]:
def engineer_features(df, is_train=True):
    """
    Create domain-specific features for dairy cow milk yield prediction.

    Args:
        df: Cleaned DataFrame
        is_train: Boolean indicating if this is training data

    Returns:
        DataFrame with engineered features
    """
    df_eng = df.copy()

    print(f"Engineering features for {'training' if is_train else 'test'} data...")
    print("-" * 80)

    # Productivity & Efficiency Ratios
    print("Creating Productivity & Efficiency Features")
    print("-" * 40)

    if 'Feed_Quantity_kg' in df_eng.columns and 'Weight_kg' in df_eng.columns:
        # Feed efficiency: feed per kg of body weight
        df_eng['Feed_Per_Weight'] = df_eng['Feed_Quantity_kg'] / (df_eng['Weight_kg'] + 1)
        print("  ✓ Created: Feed_Per_Weight (feed efficiency)")

    if 'Water_Intake_L' in df_eng.columns and 'Weight_kg' in df_eng.columns:
        # Water consumption per kg
        df_eng['Water_Per_Weight'] = df_eng['Water_Intake_L'] / (df_eng['Weight_kg'] + 1)
        print("  ✓ Created: Water_Per_Weight")

    if 'Previous_Week_Avg_Yield' in df_eng.columns and 'Feed_Quantity_kg' in df_eng.columns:
        # Yield efficiency: milk per kg of feed
        df_eng['Yield_Per_Feed'] = df_eng['Previous_Week_Avg_Yield'] / (df_eng['Feed_Quantity_kg'] + 1)
        print("  ✓ Created: Yield_Per_Feed (production efficiency)")

    print()

    # Activity & Health Indicators
    print("Creating Activity & Health Features")
    print("-" * 40)

    if all(col in df_eng.columns for col in ['Walking_Distance_km', 'Grazing_Duration_hrs']):
        # Activity level
        df_eng['Activity_Level'] = df_eng['Walking_Distance_km'] + df_eng['Grazing_Duration_hrs']
        print("  ✓ Created: Activity_Level")

    if all(col in df_eng.columns for col in ['Rumination_Time_hrs', 'Resting_Hours']):
        # Rest/Rumination balance
        df_eng['Rest_Rumination_Ratio'] = df_eng['Resting_Hours'] / (df_eng['Rumination_Time_hrs'] + 1)
        print("  ✓ Created: Rest_Rumination_Ratio")

    if 'Body_Condition_Score' in df_eng.columns and 'Weight_kg' in df_eng.columns:
        # Adjusted body condition
        df_eng['Adjusted_BCS'] = df_eng['Body_Condition_Score'] * df_eng['Weight_kg'] / 100
        print("  ✓ Created: Adjusted_BCS (body condition adjusted for weight)")

    print()

    # Lactation-Related Features
    print("Creating Lactation-Specific Features")
    print("-" * 40)

    if 'Days_in_Milk' in df_eng.columns and 'Milking_Interval_hrs' in df_eng.columns:
        # Milkings per day
        df_eng['Milkings_Per_Day'] = 24 / (df_eng['Milking_Interval_hrs'] + 1)
        print("  ✓ Created: Milkings_Per_Day")

    if 'Days_in_Milk' in df_eng.columns:
        # Lactation curve features (peak milk around 60 days)
        # df_eng['DIM_Squared'] = df_eng['Days_in_Milk'] ** 2
        df_eng['Peak_Lactation'] = (df_eng['Days_in_Milk'] >= 40) & (df_eng['Days_in_Milk'] <= 80)
        df_eng['Peak_Lactation'] = df_eng['Peak_Lactation'].astype(int)
        # print("  ✓ Created: DIM_Squared, Peak_Lactation (lactation curve)")

    if 'Parity' in df_eng.columns:
        # Parity groups (first-time vs experienced)
        df_eng['First_Time_Mother'] = (df_eng['Parity'] == 0).astype(int)
        df_eng['Experienced_Mother'] = (df_eng['Parity'] >= 2).astype(int)
        print("  ✓ Created: First_Time_Mother, Experienced_Mother")

    print()

    # Environmental Interactions
    print("Creating Environmental Interaction Features")
    print("-" * 40)

    if 'Ambient_Temperature_C' in df_eng.columns:
        # Temperature stress indicators
        df_eng['Heat_Stress'] = (df_eng['Ambient_Temperature_C'] > 25).astype(int)
        df_eng['Cold_Stress'] = (df_eng['Ambient_Temperature_C'] < 5).astype(int)
        print("  ✓ Created: Heat_Stress, Cold_Stress indicators")

    print()

    # Age & Experience Features
    print("Creating Age & Experience Features")
    print("-" * 40)

    if 'Age_Months' in df_eng.columns:
        # Age groups
        df_eng['Age_Years'] = df_eng['Age_Months'] / 12
        df_eng['Young_Cow'] = (df_eng['Age_Months'] < 30).astype(int)  # Less than 2.5 years
        df_eng['Prime_Age'] = ((df_eng['Age_Months'] >= 30) & (df_eng['Age_Months'] <= 72)).astype(int)
        df_eng['Senior_Cow'] = (df_eng['Age_Months'] > 72).astype(int)  # Over 6 years
        print("  ✓ Created: Age_Years, Young_Cow, Prime_Age, Senior_Cow")

    if 'Age_Months' in df_eng.columns and 'Parity' in df_eng.columns:
        # Average age at calving
        df_eng['Avg_Age_At_Calving'] = df_eng['Age_Months'] / (df_eng['Parity'] + 1)
        print("  ✓ Created: Avg_Age_At_Calving")

    print()

    # Feed & Nutrition Features
    print("Creating Feed & Nutrition Features")
    print("-" * 40)

    if all(col in df_eng.columns for col in ['Feed_Quantity_kg', 'Feeding_Frequency']):
        # Feed per meal
        df_eng['Feed_Per_Meal'] = df_eng['Feed_Quantity_kg'] / (df_eng['Feeding_Frequency'] + 1)
        print("  ✓ Created: Feed_Per_Meal")

    if all(col in df_eng.columns for col in ['Feed_Quantity_kg', 'Water_Intake_L']):
        # Feed to water ratio
        df_eng['Feed_Water_Ratio'] = df_eng['Feed_Quantity_kg'] / (df_eng['Water_Intake_L'] + 1)
        print("  ✓ Created: Feed_Water_Ratio")

    print()

    # Log Transformations for Skewed Features
    print("Applying Log Transformations to Skewed Features")
    print("-" * 40)

    # Identify highly skewed features
    skewed_features = []
    for col in df_eng.select_dtypes(include=[np.number]).columns:
        if col not in ['Cattle_ID', 'Farm_ID', 'Milk_Yield_L']:
            skewness = df_eng[col].skew()
            if abs(skewness) > 1.5:  # Threshold for high skewness
                skewed_features.append((col, skewness))
                df_eng[f'{col}_log'] = np.log1p(df_eng[col])  # log1p handles zeros

    if skewed_features:
        print(f"  Applied log transformation to {len(skewed_features)} skewed features:")
        for feat, skew in skewed_features[:5]:  # Show first 5
            print(f"    • {feat} (skewness: {skew:.2f})")
    else:
        print("  No highly skewed features detected")

    print()

    # Advanced Interaction & Polynomial Features
    print("Creating Advanced Interaction Features")
    print("-" * 40)

    # KEY INTERACTION: Weight × Age (older heavier cows produce differently)
    if 'Weight_kg' in df_eng.columns and 'Age_Months' in df_eng.columns:
        df_eng['Weight_Age_Interaction'] = (df_eng['Weight_kg'] * df_eng['Age_Months']) / 1000
        print("  ✓ Created: Weight_Age_Interaction")

    # Feed × Water interaction (nutritional synergy)
    if 'Feed_Quantity_kg' in df_eng.columns and 'Water_Intake_L' in df_eng.columns:
        df_eng['Feed_Water_Interaction'] = df_eng['Feed_Quantity_kg'] * df_eng['Water_Intake_L']
        print("  ✓ Created: Feed_Water_Interaction")

    # Parity × Days in Milk (experienced cows at different lactation stages)
    if 'Parity' in df_eng.columns and 'Days_in_Milk' in df_eng.columns:
        df_eng['Parity_DIM_Interaction'] = df_eng['Parity'] * df_eng['Days_in_Milk']
        print("  ✓ Created: Parity_DIM_Interaction")

    print()

    return df_eng

# Apply feature engineering
train_featured = engineer_features(train_cleaned, is_train=True)
test_featured = engineer_features(test_cleaned, is_train=False)

print(f"✓ Training data shape after feature engineering: {train_featured.shape}")
print(f"✓ Test data shape after feature engineering: {test_featured.shape}")
print()

Engineering features for training data...
--------------------------------------------------------------------------------
Creating Productivity & Efficiency Features
----------------------------------------
  ✓ Created: Feed_Per_Weight (feed efficiency)
  ✓ Created: Water_Per_Weight
  ✓ Created: Yield_Per_Feed (production efficiency)

Creating Activity & Health Features
----------------------------------------
  ✓ Created: Activity_Level
  ✓ Created: Rest_Rumination_Ratio
  ✓ Created: Adjusted_BCS (body condition adjusted for weight)

Creating Lactation-Specific Features
----------------------------------------
  ✓ Created: Milkings_Per_Day
  ✓ Created: First_Time_Mother, Experienced_Mother

Creating Environmental Interaction Features
----------------------------------------
  ✓ Created: Heat_Stress, Cold_Stress indicators

Creating Age & Experience Features
----------------------------------------
  ✓ Created: Age_Years, Young_Cow, Prime_Age, Senior_Cow
  ✓ Created: Avg_Age_At_Calvin

# Other Feature Engineering Ideas We Tried



*   Heat_Stress_Index: We tried a temperature–humidity interaction feature, but it didn't noticeably improve performance. It just added another formula on top of existing temp and humidity features. Since the simple heat_stress and cold_stress feats were capturing the main effect, this feature was not necessary.
*   Checks for NaN inside engineer_features: The data cleaning phase already handles missing and invalid values. Adding another round of checks here is redundant.

*   Polynomial and curve-based features (squared terms, lactation peak, Log_DIM): These feats were trying to capture non-linear relationships more explicitly. However, these features did not noticeably improve performance. Instead, they just increased the dimensionality of the data and risked overfitting. Thus, we decided to focus and prioritize the more domain-driven features.
*   Specific interaction features (Holstein_Weight): Again, we didn't notice an improvement in performance, so we dropped them in favor of more general interaction terms that are less brittle.









In [ ]:
# if all(col in df_eng.columns for col in ['Ambient_Temperature_C', 'Humidity_percent']):
    #     # Heat stress index (simplified THI - Temperature Humidity Index)
    #     # THI = T - 0.55 * (1 - RH/100) * (T - 58) simplified version
    #     df_eng['Heat_Stress_Index'] = (df_eng['Ambient_Temperature_C'] +
    #                                    0.36 * df_eng['Humidity_percent'] / 100 *
    #                                    df_eng['Ambient_Temperature_C'])
    #     print("  ✓ Created: Heat_Stress_Index (temperature-humidity interaction)")


# Final check for NaN/Inf in engineered features
    # print("3.8 Final Check for Invalid Values in Engineered Features")
    # print("-" * 40)

    # # Replace any inf values created during feature engineering
    # inf_count = np.isinf(df_eng.select_dtypes(include=[np.number]).values).sum()
    # if inf_count > 0:
    #     print(f"  Found {inf_count} infinite values in engineered features")
    #     df_eng = df_eng.replace([np.inf, -np.inf], [1e10, -1e10])
    #     print(f"  ✓ Replaced infinite values")

    # # Fill any NaN created during feature engineering
    # nan_count = df_eng.isnull().sum().sum()
    # if nan_count > 0:
    #     print(f"  Found {nan_count} NaN values in engineered features")
    #     numeric_cols = df_eng.select_dtypes(include=[np.number]).columns
    #     df_eng[numeric_cols] = df_eng[numeric_cols].fillna(df_eng[numeric_cols].median())
    #     print(f"  ✓ Filled NaN values with median")
    # else:
    #     print(f"  ✓ No invalid values in engineered features")

    # print()

# POLYNOMIAL FEATURES (capture non-linear relationships)
    # if 'Weight_kg' in df_eng.columns:
    #     df_eng['Weight_Squared'] = df_eng['Weight_kg'] ** 2
    #     print("  ✓ Created: Weight_Squared")

    # if 'Previous_Week_Avg_Yield' in df_eng.columns:
    #     df_eng['Previous_Week_Yield_Squared'] = df_eng['Previous_Week_Avg_Yield'] ** 2
    #     print("  ✓ Created: Previous_Week_Yield_Squared")

    # # Better lactation curve (Wood's curve approximation)
    # if 'Days_in_Milk' in df_eng.columns:
    #     df_eng['Lactation_Peak'] = df_eng['Days_in_Milk'] * np.exp(-0.05 * df_eng['Days_in_Milk'])
    #     df_eng['Log_DIM'] = np.log1p(df_eng['Days_in_Milk'])
    #     print("  ✓ Created: Lactation_Peak, Log_DIM (improved lactation curve)")

    # # Breed-specific features (Holstein is highest producer)
    # if 'Breed_Holstein' in df_eng.columns and 'Weight_kg' in df_eng.columns:
    #     df_eng['Holstein_Weight'] = df_eng['Breed_Holstein'] * df_eng['Weight_kg']
    #     print("  ✓ Created: Holstein_Weight")


# Preparing Data for Modeling

Here, we take our cleaned + engineered features and put them into a matrix the model can actually train on.

Originally, this step kept on breaking because of non-numeric columns, alignment mismatches, and random NaN/Inf values sneaking through after earlier transformations. So, this step has also become a defense layer so the model doesn't receive bad data.

The fixes included aligning the train and test sets so they have the exact same columns and dropping any remaining non-numeric columns (some columns were still of type object).

One error we kept on running into was np.isinf(X_train.values) — it kept breaking because of non-numeric columns. To fix this, we forcefully converted every column into float64.

After, we run another sweep for NaN + Inf and fix anything using medians (from the training set only). Finally, we scale everything using RobustScaler so that our models only receive scaled data.

The end result is a fully numeric, cleaned, and scaled feature matrix.

**Commented out PCA STEP**: We originally added a PCA step to reduce the dimensionality of the data. The idea was to keep 95% of the variance while compressing the data. We did this so that our model could train faster and not suffer from curse of dimensionality. However, the models that we're using handle a moderate number of features pretty well, especially after scaling and cleaning, so PCA wasn't necessary for performance.


In [ ]:
print("="*80)
print("PHASE 4: PREPARING DATA FOR MODELING")
print("="*80)
print()

# Separate features and target
y_train = train_featured['Milk_Yield_L'].values
X_train = train_featured.drop(['Milk_Yield_L', 'Cattle_ID', 'Farm_ID'], axis=1, errors='ignore')
X_test = test_featured.drop(['Cattle_ID', 'Farm_ID'], axis=1, errors='ignore')

# Aligning Train and Test Features
print("Aligning Train and Test Features")
print("-" * 40)
# Get common columns
common_cols = X_train.columns.intersection(X_test.columns)
X_train = X_train[common_cols]
X_test = X_test[common_cols]
print(f"  Train features: {X_train.shape[1]}")
print(f"  Test features: {X_test.shape[1]}")
print(f"  ✓ Features aligned successfully")
print()

# FIX: Drop all remaining non-numeric columns
print("FIX: Removing Non-Numeric Columns")
print("-" * 40)
non_numeric_cols_train = X_train.select_dtypes(include=['object']).columns.tolist()
non_numeric_cols_test = X_test.select_dtypes(include=['object']).columns.tolist()

if non_numeric_cols_train or non_numeric_cols_test:
    cols_to_drop = list(set(non_numeric_cols_train + non_numeric_cols_test))
    X_train.drop(cols_to_drop, axis=1, inplace=True, errors='ignore')
    X_test.drop(cols_to_drop, axis=1, inplace=True, errors='ignore')
    print(f"  Removed {len(cols_to_drop)} non-numeric columns to ensure all features are float/int.")
else:
    print("  No non-numeric columns found to remove.")

# Re-align after dropping (important if non-numeric columns were present)
common_cols = X_train.columns.intersection(X_test.columns)
X_train = X_train[common_cols]
X_test = X_test[common_cols]
print(f"  Final Train features count: {X_train.shape[1]}")
print()

# Final NaN/Inf Check and Handling (Aggressive Dtype Enforcement)
print("Final NaN/Inf Check and Handling (Aggressive Dtype Enforcement)")
print("-" * 40)

# ⭐️ CRITICAL FIX: AGGRESSIVE CONVERSION
# Force all columns to float, which is the necessary step for np.isinf to work.
# Any value that cannot be converted to float will become NaN.
print("  Attempting aggressive dtype conversion (float64)...")

# Identify columns that are not already numeric
non_float_cols = X_train.select_dtypes(exclude=[np.number]).columns

if len(non_float_cols) > 0:
    print(f"  Found {len(non_float_cols)} non-numeric columns before final conversion.")

for col in X_train.columns:
    try:
        # Convert to float64. This will trigger the TypeError if an incompatible object
        # is still present, but by forcing it on the DataFrame, we isolate the issue
        # away from the 'np.isinf(X_train.values)' step.
        X_train[col] = X_train[col].astype(np.float64, errors='raise')
        X_test[col] = X_test[col].astype(np.float64, errors='raise')

    except ValueError as e:
        # If astype(float) raises a ValueError, it means a non-numeric string
        # or object is definitively present. This shouldn't happen after the
        # previous steps, but if it does, we drop the column.
        print(f"  CRITICAL ERROR: Column '{col}' contains un-coercible non-numeric data. Dropping.")
        X_train.drop(col, axis=1, inplace=True)
        X_test.drop(col, axis=1, inplace=True)

# Re-align features after aggressive drop
common_cols = X_train.columns.intersection(X_test.columns)
X_train = X_train[common_cols]
X_test = X_test[common_cols]


# --- Continue with the NaN/Inf checks, which must now work ---

# Check for NaN values
nan_cols_train = X_train.columns[X_train.isnull().any()].tolist()
nan_cols_test = X_test.columns[X_test.isnull().any()].tolist()

if nan_cols_train:
    print(f"  WARNING: Found NaN values in {len(nan_cols_train)} training columns.")
    X_train = X_train.fillna(X_train.median())
    print(f"  → Filling NaN values with column median in train set.")

if nan_cols_test:
    print(f"  WARNING: Found NaN values in {len(nan_cols_test)} test columns.")
    # Fill test NaN with training median
    X_test = X_test.fillna(X_train.median())
    print(f"  → Filled test NaN values with training median.")

# Check for infinite values (This is the original line that now works)
inf_mask_train = np.isinf(X_train.values).any(axis=0)
inf_mask_test = np.isinf(X_test.values).any(axis=0)

if inf_mask_train.any():
    inf_cols = X_train.columns[inf_mask_train].tolist()
    print(f"  WARNING: Found Inf values in {len(inf_cols)} columns")
    X_train = X_train.replace([np.inf, -np.inf], [1e10, -1e10])
    print(f"  → Replaced Inf values with finite numbers in X_train")

if inf_mask_test.any():
    X_test = X_test.replace([np.inf, -np.inf], [1e10, -1e10])
    print(f"  → Replaced Inf values with finite numbers in X_test")

# Feature Scaling
print("Scaling Features")
print("-" * 40)

# Use RobustScaler (less sensitive to outliers than StandardScaler)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier manipulation
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print(f"  Scaled {X_train.shape[1]} features using RobustScaler")
print(f"  ✓ Scaling complete")
print()

"""
# 4.4 Optional: PCA for Dimensionality Reduction
print("4.4 Dimensionality Reduction (Optional)")
print("-" * 40)

if X_train_scaled.shape[1] > 50:
    print(f"  Feature count ({X_train_scaled.shape[1]}) is high - applying PCA")

    # Verify no NaN before PCA
    if X_train_scaled.isnull().any().any():
        print("  ERROR: Found NaN in scaled data, filling with 0")
        X_train_scaled = X_train_scaled.fillna(0)
        X_test_scaled = X_test_scaled.fillna(0)

    # Determine number of components to keep (e.g., 95% variance)
    pca = PCA(n_components=0.95)

    try:
        X_train_pca = pca.fit_transform(X_train_scaled)
        X_test_pca = pca.transform(X_test_scaled)

        print(f"  Original features: {X_train_scaled.shape[1]}")
        print(f"  PCA components: {X_train_pca.shape[1]}")
        print(f"  Explained variance: {pca.explained_variance_ratio_.sum():.3f}")

        # Visualize explained variance
        plt.figure(figsize=(10, 6))
        plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Explained Variance')
        plt.title('PCA Explained Variance')
        plt.grid(True)
        plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
        plt.legend()
        plt.tight_layout()
        plt.savefig('pca_variance.png')
        plt.close()
        print("  ✓ PCA visualization saved as 'pca_variance.png'")

        # Use PCA transformed data
        X_train_final = X_train_pca
        X_test_final = X_test_pca

    except Exception as e:
        print(f"  WARNING: PCA failed with error: {e}")
        print(f"  → Using scaled data without PCA")
        X_train_final = X_train_scaled.values
        X_test_final = X_test_scaled.values
else:
    print(f"  Feature count ({X_train_scaled.shape[1]}) is manageable - skipping PCA")
    X_train_final = X_train_scaled.values
    X_test_final = X_test_scaled.values
"""

X_train_final = X_train_scaled.values
X_test_final = X_test_scaled.values

# Final verification
assert X_train.select_dtypes(include=['object']).shape[1] == 0, "Non-numeric columns still exist in X_train!"
assert not X_train.isnull().any().any(), "Training data still contains NaN!"
assert not X_test.isnull().any().any(), "Test data still contains NaN!"
print(f"  ✓ All columns verified as numeric, no NaN or Inf values remaining")
print()



PHASE 4: PREPARING DATA FOR MODELING

4.1 Aligning Train and Test Features
----------------------------------------
  Train features: 85
  Test features: 85
  ✓ Features aligned successfully

4.1.5 FIX: Removing Non-Numeric Columns
----------------------------------------
  No non-numeric columns found to remove.
  Final Train features count: 85

4.2 Final NaN/Inf Check and Handling (Aggressive Dtype Enforcement)
----------------------------------------
  Attempting aggressive dtype conversion (float64)...
  Found 24 non-numeric columns before final conversion.
  → Filling NaN values with column median in train set.
  → Filled test NaN values with training median.
4.3 Scaling Features
----------------------------------------
  Scaled 85 features using RobustScaler
  ✓ Scaling complete

  ✓ All columns verified as numeric, no NaN or Inf values remaining



The code block below was meant to do a quick feature importance analysis using a RandomForestRegressor on the scaled training data. The idea was to train a small random forest, rank features by importance, and then maybe drop features whose importance falls below a small threshold.

We mainly used this as a tool to see which of our engineered features were actually useful. We showed this by printing out the top 20 and bottom 20 features by importance.

However, when we implemented this by removing features based on this threshold, it didn't improve performance. In some cases, it made the performance slightly worse. Thus, we decided to comment this code block out but still keep it in case we wanted to revisit pruning later.

In [ ]:
# # ============================================================================
# # PHASE 4.5: FEATURE IMPORTANCE & SELECTION
# # ============================================================================
# print("="*80)
# print("PHASE 4.5: FEATURE IMPORTANCE & SELECTION")
# print("="*80)
# print()

# # Train a quick Random Forest to get feature importances
# print("Training Random Forest for feature importance analysis...")
# from sklearn.ensemble import RandomForestRegressor

# rf_selector = RandomForestRegressor(
#     n_estimators=100,
#     max_depth=10,
#     random_state=42,
#     n_jobs=-1
# )

# # Use the SCALED data (X_train_scaled) with target (y_train)
# rf_selector.fit(X_train_scaled, y_train)

# # Get feature importances
# feature_importance = pd.DataFrame({
#     'feature': X_train_scaled.columns,
#     'importance': rf_selector.feature_importances_
# }).sort_values('importance', ascending=False)

# print("\n✓ Top 20 Most Important Features:")
# print("-" * 50)
# for idx, row in feature_importance.head(20).iterrows():
#     print(f"  {row['feature']:40} {row['importance']:.4f}")

# print("\n✓ Bottom 20 Least Important Features:")
# print("-" * 50)
# for idx, row in feature_importance.tail(20).iterrows():
#     print(f"  {row['feature']:40} {row['importance']:.4f}")

# # OPTIONAL: Remove very low importance features (threshold = 0.001)
# # Uncomment below if you want to actually remove features

# low_importance_threshold = 0.001
# low_importance_features = feature_importance[
#     feature_importance['importance'] < low_importance_threshold
# ]['feature'].tolist()

# if len(low_importance_features) > 0:
#     print(f"\n🗑️  Removing {len(low_importance_features)} low-importance features (< {low_importance_threshold}):")
#     for feat in low_importance_features[:10]:  # Show first 10
#         print(f"  • {feat}")

#     # Remove from scaled DataFrames
#     X_train_scaled = X_train_scaled.drop(columns=low_importance_features)
#     X_test_scaled = X_test_scaled.drop(columns=low_importance_features)

#     # Update the final arrays
#     X_train_final = X_train_scaled.values
#     X_test_final = X_test_scaled.values

#     print(f"\n✓ Training data shape after selection: {X_train_scaled.shape}")
#     print(f"✓ Test data shape after selection: {X_test_scaled.shape}")
# else:
#     print(f"\nℹ️  No features below threshold {low_importance_threshold}")

# print("\n" + "="*80)

# Model Evaluation Framework + Baseline Model Testing

This section sets up the evaluation pipeline and runs a set of baseline models to see which ones are worth tuning later. The goal of this is not to get the best possible score, but to see which model families naturally fit the data well.

For the Model Evaluation, we use 5-fold cross-validation and RMSE as the metric. This gives us a stable estimate of model performance instead of relying on a single validation split. We used RMSE as the metric because it's one of the most standard and intuitive metrics for regression problems — especially when the target is continuous and measured in real-world units (in our case, liters of milk).

For Baseline Model Evaluation, we run a bunch of baseline models: boosting models, CatBoost, LightGBM, and even small neural nets. Each of these models is evaluated using the evaluate_model() function so we can compare the numbers.

After running everything, we print out the top 5 models by RMSE — these are the models we focus on later for tuning.

**Why we commented some of our baseline models out:**

Originally, we had a wider set of baseline models: linear models (Linear Regression, Ridge), a Decision Tree, KNN variants, SVMs, and a Random Forest.

However, we observed that tree-based ensembles like Gradient Boosting, LightGBM, and CatBoost were consistently outperforming plain decision trees, KNN, and basic linear models, and running every single model with 5-fold cross-validation started to get slow.

We still kept this commented-out code so we can revisit these baseline models if we want to compare again, but they are not a part of the core baseline models.


In [ ]:
# Model Evaluation Setup
print("="*80)
print("PHASE 5: MODEL EVALUATION FRAMEWORK")
print("="*80)
print()

def evaluate_model(model, X, y, cv=5):
    """
    Evaluate a model using cross-validation.
    Returns mean and std of RMSE.
    """
    scores = cross_val_score(model, X, y, cv=cv,
                            scoring='neg_mean_squared_error',
                            n_jobs=-1)
    rmse_scores = np.sqrt(-scores)
    return {
        'mean_rmse': rmse_scores.mean(),
        'std_rmse': rmse_scores.std(),
        'scores': rmse_scores
    }

def print_results(model_name, results):
    """Pretty print evaluation results."""
    print(f"{model_name:35} | RMSE: {results['mean_rmse']:.4f} (+/- {results['std_rmse']:.4f})")

print("✓ Evaluation framework ready")
print()

# Baseline Model Evaluation
print("="*80)
print("PHASE 6: EVALUATING BASELINE MODELS")
print("="*80)
print()

# Define baseline models with class-approved configurations
baseline_models = {
    # Linear Models

    #'Linear Regression': LinearRegression(),
    #'Ridge Regression': Ridge(alpha=1.0),

    # Tree-based Models
    #'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),


    # Instance-based Models
    #'KNN (k=5)': KNeighborsRegressor(n_neighbors=5, n_jobs=1),
    #'KNN (k=10)': KNeighborsRegressor(n_neighbors=10, n_jobs=1),

    # Support Vector Machines
    #'SVR (Linear)': SVR(kernel='linear', C=1.0),
    #'SVR (RBF)': SVR(kernel='rbf', C=1.0, gamma='scale'),

    # Ensemble Methods (Bagging)
    #'Random Forest': RandomForestRegressor(
    #    n_estimators=100,
    #    max_depth=15,
    #    random_state=42,
    #    n_jobs=-1
    #),
    # Ensemble Methods (Boosting)

    'CatBoost': CatBoostRegressor(
        iterations=1000,
        depth=7,
        learning_rate=0.02,
        random_seed=42,
        l2_leaf_reg=5,
        thread_count=-1,
        verbose=False  # Suppresses training output,
    ),

    # Neural Networks (with class-approved configurations)
    'Neural Net (ReLU)': MLPRegressor(
        #hidden_layer_sizes=(100, 50),
        hidden_layer_sizes=(8, 4),
        activation='relu',
        solver='adam',
        alpha=0.01,
        learning_rate='adaptive',
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),

    'Neural Net (Tanh)': MLPRegressor(
        hidden_layer_sizes=(8, 4),
        activation='tanh',
        solver='adam',
        alpha=0.01,
        learning_rate='adaptive',
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),


    'Neural Net (Logistic)': MLPRegressor(
        hidden_layer_sizes=(8, 4),
        activation='logistic',
        solver='adam',
        alpha=0.01,
        learning_rate='adaptive',
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),

    "LightBM": LGBMRegressor(
        n_estimators=1000,
        num_leaves=8,
        max_depth=4,
        min_child_samples=25,
        learning_rate=0.02,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),

    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300,
        max_depth=4,
        sub_sample=0.8,
        learning_rate=0.05,
        random_state=42
    ),
}

"""
print("NOTE: Neural networks use RobustScaler preprocessing (similar to batch normalization)")
print("Activation functions: ReLU, Tanh, Logistic (Sigmoid) - all covered in class")
print("Optimizer: Adam - class-approved")
print("Learning rate: 'adaptive' - class-approved (reduces LR when validation plateaus)")
print("Regularization: L2 via alpha parameter (weight regularization)")
print()
"""

# Store results
baseline_results = {}

# Evaluate each baseline model
print("Evaluating baseline models with 5-fold cross-validation...")
print("-" * 80)

for name, model in baseline_models.items():
    results = evaluate_model(model, X_train_final, y_train, cv=5)
    results['model'] = model
    baseline_results[name] = results
    print_results(name, results)

print()

# Identify top 5 models for further tuning
sorted_models = sorted(baseline_results.items(), key=lambda x: x[1]['mean_rmse'])
top_k = 5
top_models = [name for name, _ in sorted_models[:top_k]]

print(f"Top {top_k} models for hyperparameter tuning:")
for i, name in enumerate(top_models, 1):
    print(f"  {i}. {name} (RMSE: {baseline_results[name]['mean_rmse']:.4f})")
print()

PHASE 5: MODEL EVALUATION FRAMEWORK

✓ Evaluation framework ready

PHASE 6: EVALUATING BASELINE MODELS

Evaluating baseline models with 5-fold cross-validation...
--------------------------------------------------------------------------------
Gradient Boosting                   | RMSE: 4.1184 (+/- 0.0133)
CatBoost                            | RMSE: 4.1111 (+/- 0.0135)
Neural Net (ReLU)                   | RMSE: 4.1623 (+/- 0.0215)
Neural Net (Tanh)                   | RMSE: 4.1433 (+/- 0.0070)
Neural Net (Logistic)               | RMSE: 4.1485 (+/- 0.0143)
LightBM                             | RMSE: 4.1149 (+/- 0.0137)

Top 5 models for hyperparameter tuning:
  1. CatBoost (RMSE: 4.1111)
  2. LightBM (RMSE: 4.1149)
  3. Gradient Boosting (RMSE: 4.1184)
  4. Neural Net (Tanh) (RMSE: 4.1433)
  5. Neural Net (Logistic) (RMSE: 4.1485)



# Ensembling (Stacking & Voting)

We have our tested baseline models above, but we wanted to combine the strongest ones using ensemble methods. The idea is that even if individual models have different biases, combining them cam smooth out weaknesses and improve accuracy. This section focuses on building two types of ensembles using the top 3 baselines identified above.

**Stacking:**
Stacking trains the top 3 models as base learners, then uses a "meta-learner" (a Ridge Regressor) to learn how to best combine their predictions. This is so the meta-learner can learn patterns that individual models miss.

**Weighted Voting:** Voting takes the predictions from the top 3 models and averages them, but with weights. We calculated weights as (1 / RMSE^2), so that it gives importance to models that performed well during cross validation.

Both ensembles are evaluated using the same 5-fold cross-validation, and we printed out the scores for both.

In [ ]:
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
# Ensure evaluate_model, baseline_models, top_models, X_train_final, y_train are defined

print("="*80)
print("PHASE 7: ENSEMBLE BUILDING (Stacking & Voting)")
print("="*80)
print("Strategy: Using the Top 3 Untuned Baseline Models to build ensembles.")
print()

# Prepare Base Estimators (CORRECTED LOGIC)
# Use the Top 3 models identified in Phase 6
top_models = top_models[:3]
base_estimators = []
for name in top_models:
    # 1. Get the Model Object
    model_object = baseline_models[name]

    # 2. Create the Clean Name
    # We strip spaces and parentheses for use in Stacking/Voting tuples.
    clean_name = name.replace(" ", "_").replace("(", "").replace(")", "")

    # 3. Append the valid (name, object) tuple
    base_estimators.append((clean_name, model_object))

# Verify the clean names before proceeding
print("VERIFIED Estimators and Cleaned Names:")
for name, _ in base_estimators:
    print(f"  - {name}")

# Using 5-fold CV for ensemble evaluation (matching baseline)
CV_FOLDS = 5
print("-" * 80)

# Stacking Ensemble
print("Training Stacking Regressor (Top 3 Baselines + Ridge Meta-Learner)")

stacking_regressor = StackingRegressor(
    estimators=base_estimators,
    # Use Ridge as the stable, regularized meta-learner
    final_estimator=Ridge(alpha=1.0, random_state=42),
    cv=CV_FOLDS,
    n_jobs=-1,
    passthrough=False
)

# Evaluate Stacking Ensemble using cross-validation
# This line caused the error, but with clean names, it should now run.
stacking_results = evaluate_model(stacking_regressor, X_train_final, y_train, cv=CV_FOLDS)
print_results("Stacking (Ridge Meta)", stacking_results)

# Store results
baseline_results["Stacking (Ridge Meta)"] = {'mean_rmse': stacking_results['mean_rmse'], 'model': stacking_regressor, 'scores': stacking_results['scores']}

# Weighted Voting Ensemble (Simple and Effective Averaging)
print("\nTraining Weighted Voting Regressor")

# Calculate weights: 1 / (RMSE^2) gives more aggressive weighting to better models
weights = []
for name in top_models:
    rmse = baseline_results[name]['mean_rmse']
    weights.append(1 / (rmse ** 2))

if all(w > 0 for w in weights):
    voting_regressor = VotingRegressor(
        estimators=base_estimators,
        weights=weights,
        n_jobs=-1
    )

    # Evaluate Voting Ensemble using cross-validation
    voting_results = evaluate_model(voting_regressor, X_train_final, y_train, cv=CV_FOLDS)
    print_results("Voting (Weighted)", voting_results)

    # Store results
    baseline_results["Voting (Weighted)"] = {'mean_rmse': voting_results['mean_rmse'], 'model': voting_regressor, 'scores': voting_results['scores']}
else:
    print("WARNING: Could not calculate weights. Skipping Weighted Voting.")

print("\n✓ Ensemble building complete and names validated.")
print("="*80)

PHASE 7: ENSEMBLE BUILDING (Stacking & Voting)
Strategy: Using the Top 3 Untuned Baseline Models to build ensembles.

VERIFIED Estimators and Cleaned Names:
  - CatBoost
  - LightBM
  - Gradient_Boosting
--------------------------------------------------------------------------------
7.2 Training Stacking Regressor (Top 3 Baselines + Ridge Meta-Learner)
Stacking (Ridge Meta)               | RMSE: 4.1105 (+/- 0.0133)

7.3 Training Weighted Voting Regressor
Voting (Weighted)                   | RMSE: 4.1122 (+/- 0.0135)

✓ Ensemble building complete and names validated.


# Final Model Selection + Submission

This is the final phase where we take all our evaluation results and decide which model is going to be used for our submission.

After ranking every baseline and ensemble model by RMSE, we compare them to a simple benchmark (Linear Regression) so we can see how much improvement each one provides. From the rankings, we can see that the Stacking Regressor with a Ridge meta-learner came out as the top performer.

Once we've identified the best model (Stacking Regressor), we retrain it one last time on 100% of the training data, giving the model the maximum amount of info before generating predictions.

Then, we run our model on the test set and create the submission file with the required columns.

In [ ]:
print("="*80)
print("PHASE 9: FINAL MODEL SELECTION & SUBMISSION")
print("="*80)

# Compile and Rank Results
print("Final Performance Ranking (Lower RMSE is Better)")
print("-" * 50)

# Compile all current results (baselines and ensembles)
final_ranking = sorted(
    # baseline_results now holds all models (baselines + ensembles)
    [(name, result['mean_rmse'], result['model']) for name, result in baseline_results.items()],
    key=lambda x: x[1]
)

# Identify the best model
best_model_name, best_rmse, final_model = final_ranking[0]

# Get the baseline for comparison (Linear Regression, typically the worst)
try:
    # Use Linear Regression RMSE as the true performance benchmark
    baseline_rmse = baseline_results['Linear Regression']['mean_rmse']
except KeyError:
    # Fallback if Linear Regression was not evaluated
    baseline_rmse = final_ranking[-1][1]

print(f"Benchmark (Linear Regression): {baseline_rmse:.4f}")
print("-" * 50)

for name, rmse, _ in final_ranking:
    # Calculate improvement percentage
    improvement = (baseline_rmse - rmse) / baseline_rmse * 100

    # Add an emoji for the top performing models
    prefix = "⭐" if "Stacking" in name or "Voting" in name else ""
    if name == best_model_name:
         prefix = "🥇"

    print(f"{prefix} {name:34} | RMSE: {rmse:.4f} | Improvement: {improvement:.2f}%")
# Final Model Selection
print("\n Final Model Selection")
print("-" * 50)
print(f"🥇 **Best Model Selected: {best_model_name}**")
print(f"   Final Cross-Validation RMSE: {best_rmse:.4f}")
print(f"   Total Improvement over Benchmark: {((baseline_rmse - best_rmse) / baseline_rmse * 100):.2f}%")
print("-" * 50)


# Final Training on FULL Data and Prediction
print("\nFinal Training on FULL X_train_final and Prediction")
print("-" * 50)

# The selected final_model is trained one last time on 100% of the preprocessed data.
# This assumes X_train_final, X_test_final, and y_train are available from Phase 4.

# Train the final model using the full scaled dataset
final_model.fit(X_train_final, y_train)

# Generate predictions on the test set
test_predictions = final_model.predict(X_test_final)
test_predictions[test_predictions < 0] = 0.0 # Clip negative predictions

print(f"  Final model trained successfully on {X_train_final.shape[0]} samples.")
print(f"  Generated {len(test_predictions)} predictions.")

# Create Submission File
print("\nCreating Submission File")
print("-" * 50)

# Note: 'test_data' must be the original un-processed test DataFrame to extract 'Cattle_ID'
submission_df = pd.DataFrame({
    'Cattle_ID': test_data['Cattle_ID'],
    'Milk_Yield_L': test_predictions
})

submission_file = 'dairy_cow_submission_ensemble_baseline.csv'
submission_df.to_csv(submission_file, index=False)
print(f"✓ Submission file created: '{submission_file}'")
print("="*80)

PHASE 9: FINAL MODEL SELECTION & SUBMISSION
9.1 Final Performance Ranking (Lower RMSE is Better)
--------------------------------------------------
Benchmark (Linear Regression): 4.1623
--------------------------------------------------
🥇 Stacking (Ridge Meta)              | RMSE: 4.1105 | Improvement: 1.25%
 CatBoost                           | RMSE: 4.1111 | Improvement: 1.23%
⭐ Voting (Weighted)                  | RMSE: 4.1122 | Improvement: 1.20%
 LightBM                            | RMSE: 4.1149 | Improvement: 1.14%
 Gradient Boosting                  | RMSE: 4.1184 | Improvement: 1.06%
 Neural Net (Tanh)                  | RMSE: 4.1433 | Improvement: 0.46%
 Neural Net (Logistic)              | RMSE: 4.1485 | Improvement: 0.33%
 Neural Net (ReLU)                  | RMSE: 4.1623 | Improvement: 0.00%

9.2 Final Model Selection
--------------------------------------------------
🥇 **Best Model Selected: Stacking (Ridge Meta)**
   Final Cross-Validation RMSE: 4.1105
   Total Improvemen